## verificar dimansion de las imágenes

In [1]:
import os
from PIL import Image

def check_image_sizes(folder_path):
    # Tamaño esperado de las imágenes
    expected_size = (1024, 1024)
    
    # Obtener la lista de archivos en la carpeta
    files = os.listdir(folder_path)
    
    # Filtrar solo los archivos con extensión .png
    png_files = [file for file in files if file.lower().endswith('.png')]
    
    # Inicializar una lista para almacenar los resultados
    incorrect_size_images = []
    
    # Verificar cada imagen PNG
    for file in png_files:
        image_path = os.path.join(folder_path, file)
        
        try:
            with Image.open(image_path) as img:
                if img.size != expected_size:
                    incorrect_size_images.append((file, img.size))
        except Exception as e:
            print(f"Error al abrir la imagen {file}: {e}")
    
    # Resultados
    if incorrect_size_images:
        print("Las siguientes imágenes no tienen el tamaño 1024x1024:")
        for image_info in incorrect_size_images:
            print(f"{image_info[0]} - Tamaño: {image_info[1]}")
    else:
        print("Todas las imágenes tienen el tamaño 1024x1024.")

# Ejemplo de uso
check_image_sizes("./7_dataset_normalizado_ajustado")


Todas las imágenes tienen el tamaño 1024x1024.


## Renombrar las imágenes y clasificarlas

In [2]:
import os
import pandas as pd
import shutil

# Ruta de la carpeta donde están las imágenes PNG
input_dir = "./7_dataset_normalizado_ajustado"
output_dir = "./8_dataset_clasificado_BI-RADS_prueba"

# Lista con las rutas de los archivos CSV que contienen los nombres
archivos_csv = [
    "CC_BI-RADS_1_prueba.csv",
    "MLO_BI-RADS_1_prueba.csv",
    "CC_BI-RADS_2_prueba.csv",
    "MLO_BI-RADS_2_prueba.csv",
    "CC_BI-RADS_3_prueba.csv",
    "MLO_BI-RADS_3_prueba.csv",
    "CC_BI-RADS_4_prueba.csv",
    "MLO_BI-RADS_4_prueba.csv",
    "CC_BI-RADS_5_prueba.csv",
    "MLO_BI-RADS_5_prueba.csv"
]

# Función para renombrar las imágenes y moverlas a una subcarpeta correspondiente
def renombrar_y_mover_imagenes(carpeta_entrada, carpeta_salida, dataframe, nombre_csv):
    # Crear una subcarpeta basada en el nombre del archivo CSV sin la parte "_prueba"
    nombre_subcarpeta = os.path.splitext(os.path.basename(nombre_csv))[0].replace("_prueba", "")
    subcarpeta = os.path.join(carpeta_salida, nombre_subcarpeta)
    
    if not os.path.exists(subcarpeta):
        os.makedirs(subcarpeta)
    
    for index, row in dataframe.iterrows():
        original_name = row[0]  # Primera columna es el nombre original
        new_name = row[1]       # Segunda columna es el nuevo nombre (incluye .png)
        
        # Agregar la extensión .png si no está presente en el nombre original
        if not original_name.endswith('.png'):
            original_name = f"{original_name}.png"
        
        # Ruta completa del archivo original
        archivo_original = os.path.join(carpeta_entrada, original_name)
        # Ruta del archivo en la subcarpeta con el nuevo nombre
        archivo_final = os.path.join(subcarpeta, new_name)
        
        # Verifica si el archivo original existe antes de copiar
        if os.path.exists(archivo_original):
            try:
                # Copiar el archivo y renombrarlo en la carpeta de salida
                shutil.copy(archivo_original, archivo_final)
                print(f"Archivo {original_name} copiado y renombrado como {new_name} en {subcarpeta}")
            except Exception as e:
                print(f"Error al copiar {original_name}: {e}")
        else:
            print(f"El archivo {original_name} no existe en la carpeta de entrada.")

# Procesa todos los archivos CSV en la lista
for archivo_csv in archivos_csv:
    # Lee cada archivo CSV sin encabezado
    df = pd.read_csv(archivo_csv, header=None)
    # Renombra y mueve las imágenes basadas en el CSV actual
    renombrar_y_mover_imagenes(input_dir, output_dir, df, archivo_csv)

print("Todos los archivos han sido renombrados y movidos a sus respectivas subcarpetas.")

Archivo df26762e7c847377fb42346b2c7c1bed.png copiado y renombrado como b1ea3a30c883b0e10b1e1a4afc2fc55d_L.png en ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1
Archivo 6005c4c1b5532626ec05540117b230a6.png copiado y renombrado como b1ea3a30c883b0e10b1e1a4afc2fc55d_R.png en ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1
Archivo 4f4dd6a388dad534024009670c3f7d0e.png copiado y renombrado como 76c9da038078c93d58565d20706258cc_L.png en ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1
Archivo 530eceeb7639d0b27a80da57fd1ca6a5.png copiado y renombrado como 76c9da038078c93d58565d20706258cc_R.png en ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1
Archivo 24168c3c58fb7df80390c6acf8c26071.png copiado y renombrado como 83f1408ca03115e775c7a059a0c9663e_L.png en ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1
Archivo 5dd10ba0cd1f0d0bf1011c3030412451.png copiado y renombrado como 83f1408ca03115e775c7a059a0c9663e_R.png en ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1
Archivo d7

## organizar las carpetas en subcarpetas CC/MLO y train/test

In [3]:
import os
import shutil

# Función para mover o copiar las imágenes a las subcarpetas correspondientes en CC y MLO y mantener la clasificación BI-RADS
def mover_a_subcarpetas(source_dir, target_dir_cc, target_dir_mlo):
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.png'):
                # Verificamos la carpeta de origen (root) para saber si es CC o MLO y si es train o test
                if "CC" in root:
                    # Identificamos el BI-RADS
                    for birads in ['BI-RADS_1', 'BI-RADS_2', 'BI-RADS_3', 'BI-RADS_4', 'BI-RADS_5']:
                        if birads in root:
                            dest_dir = os.path.join(target_dir_cc, birads)
                            # Crear la carpeta si no existe
                            os.makedirs(dest_dir, exist_ok=True)
                            shutil.copy(os.path.join(root, file), os.path.join(dest_dir, file))
                            print(f'Copiando {file} desde {root} a {dest_dir}')
                elif "MLO" in root:
                    # Identificamos el BI-RADS
                    for birads in ['BI-RADS_1', 'BI-RADS_2', 'BI-RADS_3', 'BI-RADS_4', 'BI-RADS_5']:
                        if birads in root:
                            dest_dir = os.path.join(target_dir_mlo, birads)
                            # Crear la carpeta si no existe
                            os.makedirs(dest_dir, exist_ok=True)
                            shutil.copy(os.path.join(root, file), os.path.join(dest_dir, file))
                            print(f'Copiando {file} desde {root} a {dest_dir}')

# Función principal para crear la estructura CC y MLO, y clasificar imágenes
def clasificar_en_cc_mlo(source_dir, output_dir):
    # Carpetas de salida CC y MLO dentro de 10_dataset_classified
    cc_dir = os.path.join(output_dir, 'CC')
    mlo_dir = os.path.join(output_dir, 'MLO')

    # Crear carpetas CC y MLO si no existen
    os.makedirs(cc_dir, exist_ok=True)
    os.makedirs(mlo_dir, exist_ok=True)

    # Llamar a la función para mover las imágenes a las carpetas correspondientes
    mover_a_subcarpetas(source_dir, cc_dir, mlo_dir)

    print("Clasificación en subcarpetas CC y MLO completada.")

# Directorio de entrada (carpeta 9_dataset)
source_dir = './8_dataset_clasificado_BI-RADS_prueba'

# Directorio de salida (donde se organizarán las imágenes en CC y MLO)
output_dir = './Dataset_prueba'

# Clasificar las imágenes en las subcarpetas CC y MLO
clasificar_en_cc_mlo(source_dir, output_dir)

Copiando 003700f3c960e0b9bca2b8437c3dbf05_R.png desde ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1 a ./Dataset_prueba\CC\BI-RADS_1
Copiando 004426a40da27ef22a866538b772ac44_L.png desde ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1 a ./Dataset_prueba\CC\BI-RADS_1
Copiando 004426a40da27ef22a866538b772ac44_R.png desde ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1 a ./Dataset_prueba\CC\BI-RADS_1
Copiando 006f2e9cb3256c02fe82a7ff29af6260_L.png desde ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1 a ./Dataset_prueba\CC\BI-RADS_1
Copiando 00a369b4ec1e5e0ff34e6bd838e5f2d6_R.png desde ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1 a ./Dataset_prueba\CC\BI-RADS_1
Copiando 00ba2f2c0cd9a34af770eb252f3a56ce_L.png desde ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1 a ./Dataset_prueba\CC\BI-RADS_1
Copiando 00ba2f2c0cd9a34af770eb252f3a56ce_R.png desde ./8_dataset_clasificado_BI-RADS_prueba\CC_BI-RADS_1 a ./Dataset_prueba\CC\BI-RADS_1
Copiando 00c3c05f7ff415d71fae16ae9